In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [150]:
import glob
import os
import json
import numpy as np
import re
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from collections import OrderedDict, defaultdict
from functools import lru_cache
from ipdb import set_trace
from tqdm import tqdm_notebook as tqdm
from tabulate import tabulate

from allennlp.data import Instance, Token
from allennlp.data.fields import TextField, ListField, MetadataField, ArrayField, IndexField, Field, AdjacencyField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import PretrainedBertIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers.word_splitter import BertBasicWordSplitter
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.iterators import BasicIterator
from allennlp.models import Model, SimpleSeq2Seq
from allennlp.modules.token_embedders import PretrainedBertEmbedder
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.training import Trainer
from allennlp.training.moving_average import ExponentialMovingAverage
from allennlp.training.metrics import CategoricalAccuracy, BooleanAccuracy
from allennlp.nn.util import get_text_field_mask, move_to_device

In [2]:
np.random.seed(1)

In [3]:
class Schema(object):

    def __init__(self, filepath):
        with open(filepath) as f:
            self.index = {}
            for schema in json.load(f):
                service_name = schema["service_name"]
                self.index[service_name] = schema

    @lru_cache(maxsize=None)
    def get(self, service):
        result = dict(
            # service
            name=service,
            desc=self.index[service]["description"],
            
            # slots
            slot_name=[],
            slot_desc=[],
            slot_iscat=[], 
            slot_vals=[], # collected only for cat slots.. not sure if that makes sense

            # intents
            intent_name=[],
            intent_desc=[],
            intent_istrans=[],
            intent_reqslots=[],
            intent_optslots=[],
            intent_optvals=[],
        )

        for slot in self.index[service]["slots"]:
            result["slot_name"].append(slot["name"])
            result["slot_desc"].append(slot["description"])
            result["slot_iscat"].append(slot["is_categorical"])
            result["slot_vals"].append(slot["possible_values"])
        
        for intent in self.index[service]["intents"]:
            result["intent_name"].append(intent["name"])
            result["intent_desc"].append(intent["description"])
            result["intent_istrans"].append(intent["is_transactional"])
            result["intent_reqslots"].append(intent["required_slots"])
            result["intent_optslots"].append(list(intent["optional_slots"].keys()))
            result["intent_optvals"].append(list(intent["optional_slots"].values()))

        return result

In [96]:
class Memory(object):
    
    def __init__(self, schema, services):
        self.schema = schema
        
        # memory for cat slots: serv,slot=>[] or for noncat slots: noncat => []
        self.memory = defaultdict(list)
        self.index = defaultdict(set) # serv,slot->val, noncat->val

        for serv in services:
            # add possible values from slot
            sch = schema.get(serv)
            for slot, iscat, slotvals in zip(sch["slot_name"], sch["slot_iscat"], sch["slot_vals"]):
                key = (serv, slot) if iscat else "noncat"
                for val in ["NONE", "dontcare"] + slotvals:
                    if val not in self.index[key]:
                        self.index[key].add(val)
                        self.memory[key].append(val)

            # add optional slot vals
            for optslots, optvals in zip(sch["intent_optslots"], sch["intent_optvals"]):
                for slot, val in zip(optslots, optvals):
                    slotid = sch["slot_name"].index(slot)
                    iscat = sch["slot_iscat"][slotid]
                    assert slotid != -1
                    key = (serv, slot) if iscat else "noncat"
                    if val not in self.index[key]:
                        self.index[key].add(val)
                        self.memory[key].append(val)
                        
    def update(self, dial_turn):
        # update only noncat values..
        utter = dial_turn["utterance"]
        for frame in dial_turn["frames"]:
            sch = self.schema.get(frame["service"])
            slot_names = sch["slot_name"]
            slot_iscat = sch["slot_iscat"]

            for tag in frame["slots"]:
                slot, st, en = tag["slot"], tag["start"], tag["exclusive_end"]
                slotid = slot_names.index(slot)
                iscat = slot_iscat[slotid]
                assert slotid != -1

                if not iscat:
                    value = utter[st:en]
                    key = "noncat"
                    if value not in self.index[key]:
                        value = re.sub("\u2013", "-", value) # dial 59_00125 turn 14
                        self.index[key].add(value)
                        self.memory[key].append(value)
                        
    def get(self, key="noncat"):
        return self.memory[key]
    
    
class DialogReader(DatasetReader):

    def __init__(self, schema, limit, lazy=False):
        super().__init__(lazy)
        self.token_indexers = {"tokens": PretrainedBertIndexer("bert-base-uncased")}
        self.tokenizer = BertBasicWordSplitter()
        self.schema = schema
        self.limit = limit

    def _read(self, path):
        # get a set of dialogs
        count = 0
        dialogs = []
        for filename in sorted(glob.glob(path)):
            if count > self.limit:
                break
            with open(filename) as f:
                for d in json.load(f):
                    dialogs.append(d)
                    count += 1
                    if count > self.limit:
                        break
        
        # prepare instances
        for dial in dialogs:
            memory = Memory(self.schema, dial["services"])
            for turnid, turn in enumerate(dial["turns"]):
                memory.update(turn)
                if turn["speaker"] == "USER":
                    usr_utter = turn["utterance"]
                    sys_utter = dial["turns"][turnid-1]["utterance"] if turnid > 0 else "dialog started"
                    num_none_questions  = 0
                    
                    for frame in turn["frames"]:
                        # get schema info
                        serv = frame["service"]
                        sch = self.schema.get(serv)
                        
                        # intent
                        intent = frame["state"]["active_intent"]
                        all_intents = {s: i for i, s in enumerate(sch["intent_name"])}
                        intent_istrans = False
                        intent_desc = "No intent"
                        if intent != "NONE":
                            intentid = all_intents[intent]
                            assert intentid != -1
                            intent_desc = sch["intent_desc"][intentid]
                            intent_istrans = sch["intent_istrans"][intentid]
                        
                        # slots
                        all_slots = {s: i for i, s in enumerate(sch["slot_name"])}
                        all_slots_iscat = sch["slot_iscat"]
                        all_slots_desc = sch["slot_desc"]
                        active_slots = frame["state"]["slot_values"]
                        none_slots = set(all_slots) - set(active_slots)
                        
                        # active slots
                        for slot, values in active_slots.items():
                            slotid = all_slots[slot]
                            assert slotid != -1
                            key = (serv, slot) if all_slots_iscat[slotid] else "noncat"
                            target_value = re.sub("\u2013", "-", values[0])
                            
                            item = dict(
                                dialid=dial["dialogue_id"],
                                turnid=turnid,
                                usr_utter=usr_utter,
                                sys_utter=sys_utter,
                                serv=serv,
                                serv_desc=sch["desc"],
                                slot=slot,
                                slot_desc=all_slots_desc[slotid],
                                slot_iscat=all_slots_iscat[slotid],
                                slot_val=target_value,
                                intent=intent,
                                intent_desc=intent_desc,
                                intent_istrans=intent_istrans,
                                memory=memory.get(key),
                            )
                            yield self.text_to_instance(item)
                            
                        # none valued slots
                        for slot in none_slots:
                            if np.random.randn() > 0.5 and num_none_questions < 3:
                                num_none_questions += 1
                                slotid = all_slots[slot]
                                assert slotid != -1
                                key = (serv, slot) if all_slots_iscat[slotid] else "noncat"
                                target_value = "NONE"
                                item = dict(
                                    dialid=dial["dialogue_id"],
                                    turnid=turnid,
                                    usr_utter=usr_utter,
                                    sys_utter=sys_utter,
                                    serv=serv,
                                    serv_desc=sch["desc"],
                                    slot=slot,
                                    slot_desc=all_slots_desc[slotid],
                                    slot_iscat=all_slots_iscat[slotid],
                                    slot_val=target_value,
                                    intent=intent,
                                    intent_desc=intent_desc,
                                    intent_istrans=intent_istrans,
                                    memory=memory.get(key),
                                )
                                yield self.text_to_instance(item)
                        
            
    def text_to_instance(self, item):
        fields = {}
        
        # featurize query
        query_tokens = []
        query_type = []
        
        for index, field in enumerate(("sys_utter", "usr_utter", "serv_desc", "slot_desc")):
            tokens = self.tokenizer.split_words(item[field])
            query_tokens.extend(tokens)
            query_type.extend([index + 1] * len(tokens))
        
        query_pos = list(range(1, len(query_tokens) + 1))
        
        fields["query"] = TextField(query_tokens, self.token_indexers)
        fields["query_type"] = ArrayField(np.array(query_type))
        fields["query_pos"] = ArrayField(np.array(query_pos))
        
        # featurize memory
        mem_values = item["memory"]
        mem_tokens = []
        mem_loc = []
        mem_type = []
        mem_pos = []
        
        for index, mem_val in enumerate(mem_values):
            tokens = self.tokenizer.split_words(mem_val)
            loc = int(mem_val == item["slot_val"])
            for i, tok in enumerate(tokens):
                mem_tokens.append(tok)
                mem_loc.append(loc * (i == len(tokens)-1))
                mem_type.append(index + 1)
            
        mem_pos = list(range(1, len(mem_tokens) + 1))
        
        fields["memory"] = TextField(mem_tokens, self.token_indexers)
        fields["memory_loc"] = ArrayField(np.array(mem_loc), padding_value=-1)
        fields["memory_type"] = ArrayField(np.array(mem_type))
        fields["memory_pos"] = ArrayField(np.array(mem_pos))
        
        # positional fields
        fields["turnid"] =  ArrayField(np.array(item["turnid"]))
        
        # meta fields
        fields["id"] = MetadataField("{}/{}/{}/{}".format(item["dialid"], item["turnid"], item["serv"], item["slot"]))
        fields["slot"] = MetadataField(item["slot"])
        fields["serv"] = MetadataField(item["serv"])
        fields["intent"] = MetadataField(item["intent"])
        fields["dialid"] = MetadataField(item["dialid"])
        fields["memory_values"] = MetadataField(item["memory"])
        
        return Instance(fields)

In [46]:
train_schema = Schema("../data/train/schema.json")
dev_schema = Schema("../data/dev/schema.json")

In [97]:
# read full dataset
reader = DialogReader(train_schema, limit=100)
train_ds = reader.read("../data/train/dialogues*.json")

reader = DialogReader(dev_schema, limit=5)
dev_ds = reader.read("../data/dev/dialogues*.json")

vocab = Vocabulary.from_instances(train_ds + dev_ds)


it = BasicIterator(batch_size=32)
it.index_with(vocab)
batch = next(iter(it(train_ds)))
batch.keys()

dict_keys(['query', 'query_type', 'query_pos', 'memory', 'memory_loc', 'memory_type', 'memory_pos', 'turnid', 'id', 'slot', 'serv', 'intent', 'dialid', 'memory_values'])

In [98]:
batch["memory_loc"][5]

tensor([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1.])

In [109]:
class CandidateSelector(Model):
    
    def __init__(self, vocab):
        super().__init__(vocab)
        self.emb = PretrainedBertEmbedder("bert-base-uncased", requires_grad=True)
        emb_dim = self.emb.get_output_dim()
        
        # query encoder
        self.register_buffer("query_state_history", None)
        self.register_buffer("query_state", None)
        self.query_enc = nn.GRU(3 * emb_dim, emb_dim, batch_first=True, bidirectional=True)
        self.query_type_emb = nn.Embedding(10, emb_dim) # max query fields
        self.query_pos_emb = nn.Embedding(500, emb_dim) # max query length
        
        # candidate values decoder
        self.memory_dec = nn.GRU(4 * emb_dim + 1, emb_dim, batch_first=True, num_layers=2, dropout=0.2)
        self.memory_type_emb = nn.Embedding(100, emb_dim) # max candidate values
        self.memory_pos_emb = nn.Embedding(500, emb_dim) # max total length of candidates
        
        # query attention
        self.attn_w = nn.Linear(4 * emb_dim, emb_dim)
        self.attn_v = nn.Linear(emb_dim, 1, bias=False)
        
        # final state
        self.final = nn.Linear(emb_dim, 1)
        
        self.accuracy = BooleanAccuracy()
        self.dropout = nn.Dropout(p=0.3)
        
        #self.init_weights()
        
    def init_weights(self):
        for n, p in self.named_parameters():
            if not n.startswith(("emb",)):
                if "weight" in n:
                    p.data.normal_(mean=0, std=1)
                else:
                    p.data.zero_()
        
    def get_metrics(self, reset=False):
        return {"acc": self.accuracy.get_metric(reset)}
    
    def encoder(self, inputs, reset=False):
        hidden = self.query_state
        if reset:
            hidden = None
        
        # if hidden size is uneven
        if hidden is not None:
            hidden_bs = hidden.shape[1]
            inputs_bs = inputs.shape[0]
            if hidden_bs > inputs_bs:
                hidden = hidden[:,:inputs_bs,:]
            elif hidden_bs < inputs_bs:
                padding = torch.zeros(
                    hidden.shape[0], inputs_bs-hidden_bs, hidden.shape[2],
                    device=hidden.device, dtype=hidden.dtype)
                hidden = torch.cat([hidden, padding], dim=1)
            hidden = hidden.contiguous()
        
        # forward pass
        outputs, hidden = self.query_enc(inputs, hidden)
        
        # retain query state
        self.query_state = hidden.detach()
        
        return outputs, hidden
    
    def query_state_attention(self, mem_h):
        # mem_h -- [layers * dir, batch, emb]
        # query_o -- [batch, seq, dir * emb]
        sh = mem_h.shape
        
        energies = []
        
        queries_o = self.query_state_history
        for x in range(3 - len(queries_o)):
            empty_query = torch.zeros()
        
        sh = mem_h.shape
        mem_h = mem_h.view(sh[1], 1, -1) # b,1,lde
        mem_h = mem_h.repeat(1, query_o.shape[1], 1) # b,s,lde
        
        x = torch.cat((query_o, mem_h), -1) # b,s, l*d*e + d*e
        energy = torch.tanh(self.attn_w(x)) # bse
        energy = self.attn_v(energy).squeeze(-1) # bs1
        
        attn = F.softmax(energy, -1) # bs
        
        # context
        if self.query_enc.bidirectional:
            sh = query_o.shape[-1] // 2
            query_o = query_o[...,:sh] + query_o[...,sh:]
            
        context = torch.bmm(attn.unsqueeze(1), query_o) # b1e
        
        return context # b1e
    
    def query_attention(self, mem_h, query_o):
        # mem_h -- [layers * dir, batch, emb]
        # query_o -- [batch, seq, dir * emb]
        energies = []
        
        sh = mem_h.shape
        mem_h = mem_h.view(sh[1], 1, -1) # b,1,lde
        mem_h = mem_h.repeat(1, query_o.shape[1], 1) # b,s,lde
        
        x = torch.cat((query_o, mem_h), -1) # b,s, l*d*e + d*e
        energy = torch.tanh(self.attn_w(x)) # bse
        energy = self.attn_v(energy).squeeze(-1) # bs1
        
        attn = F.softmax(energy, -1) # bs
        
        # context
        if self.query_enc.bidirectional:
            sh = query_o.shape[-1] // 2
            query_o = query_o[...,:sh] + query_o[...,sh:]
            
        context = torch.bmm(attn.unsqueeze(1), query_o) # b1e
        
        return context # b1e
    
    def decoder(self, query_o, query_h, mem_inp, mem_out):
        # query_o -- [batch, seq, dir*emb]
        # query_h -- [layers * dir, batch, emb]
        # mem_inp -- [batch, mem, emb]
        # mem_out -- [batch, mem]
        predicted = []
        
        # init prev final out
        bs = mem_inp.shape[0]
        prev_out = torch.zeros(bs, 1, 1, device=mem_inp.device, dtype=mem_inp.dtype) # b11
        
        # init decoder hidden state
        layers = self.query_enc.num_layers 
        if self.query_enc.bidirectional:
            dec_h = query_h[:layers,...] + query_h[layers:,...]
        if layers > 1:
            dec_h = query_h.sum(0, keepdim=True)
            
        dim = self.memory_dec.num_layers * (1 + int(self.memory_dec.bidirectional))
        dec_h = dec_h.repeat(dim, 1, 1) # [layers * dir, batch, emb]
        
        # predict at each candidate value
        for m in range(mem_inp.shape[1]):
            inp = mem_inp[:,m:m+1,:] # b,1,3e
            context = self.query_attention(dec_h, query_o) # b1,e
            combined_inp = torch.cat((inp, context, prev_out), dim=-1) # b,1,4e+1
            
            dec_o, dec_h = self.memory_dec(combined_inp, dec_h)
            
            predicted.append(dec_o)
            prev_out = mem_out[:,m:m+1,None] # teacher forcing
        
        predicted = torch.cat(predicted, dim=1) # bme
        
        return predicted
    
    def forward(self, **batch):
        query = self.emb(batch["query"]["tokens"], batch["query"]["tokens-offsets"]) # [batch, seq, emb]
        query_type = self.query_type_emb(batch["query_type"].long()) # [batch, seq, emb]
        query_pos = self.query_pos_emb(batch["query_pos"].long()) # [batch, seq, emb]
        
        memory = self.emb(batch["memory"]["tokens"], batch["memory"]["tokens-offsets"]) # [batch, mem, emb]
        memory_type = self.memory_type_emb(batch["memory_type"].long()) # [batch, mem, emb]
        memory_pos = self.memory_pos_emb(batch["memory_pos"].long())
        memory_loc_oh = batch["memory_loc"] # [batch, mem]
        
        is_new_dial = bool((batch["turnid"] == 0).all())
        query_i = torch.cat((query, query_type, query_pos), -1) # [batch, seq, 3*emb]
        query_o, query_h = self.encoder(query_i, reset=is_new_dial) # [batch, seq, dir * emb], [layers * dir, batch, emb]
        
        # decode values
        memory_i = torch.cat((memory, memory_type, memory_pos), dim=-1) # [batch, seq, 3*emb]
        mem_o = self.decoder(query_o, query_h, memory_i, memory_loc_oh) # [batch, mem, emb]
        predicted = F.softmax(self.final(mem_o).squeeze(-1), -1) # [batch, mem]
        
        # set NANs
        if (predicted != predicted).any():
            predicted[predicted != predicted] = 0
            print("Warning: found NaN")

        # loss
        target_loc = memory_loc_oh.argmax(-1)
        loss = F.cross_entropy(predicted, target_loc, ignore_index=-1).unsqueeze(0)
            
        # metric
        predicted_loc = predicted.argmax(-1)
        target_loc = memory_loc_oh.argmax(-1)

        mask = batch["memory"]["mask"]
        mask = mask[torch.arange(mask.shape[0]), target_loc]

        self.accuracy(predicted_loc, target_loc, mask)

        output = dict(
            logits=predicted,
            loss=loss,
            pred=predicted_loc,
            target=target_loc,
        )
        
        return output

In [110]:
# litmus test on CPU
model = CandidateSelector(vocab).to("cpu")
batch=move_to_device(batch, -1)
model(**batch)

In [112]:
allen_device=1
torch_device=1

model = CandidateSelector(vocab).to(torch_device)
optimizer = optim.Adam(model.parameters(), lr=3e-5)

iterator = BasicIterator(batch_size=32)
iterator.index_with(vocab)


trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    num_epochs=2,
    cuda_device=allen_device,
    serialization_dir="../results/4",
    should_log_learning_rate=True,
    histogram_interval=50,
    num_serialized_models_to_keep=1,
    grad_norm=1,
    shuffle=False,
)

trainer.train()

unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/suryak/anaconda3/lib/python3.7/site-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 395, in check_output
    **kwargs).stdout
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 472, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 775, in __init__
    restore_signals, start_new_session)
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 1453, in _execute_child
    restore_signals, start_new_session, preexec_fn)
OSError: [Errno 12] Cannot allocate memory


unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/home/suryak/anaconda3/lib/python3.7/site-packages/allennlp/common/util.py", line 378, in gpu_memory_mb
    encoding='utf-8')
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 395, in check_output
    **kwargs).stdout
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 472, in run
    with Popen(*popenargs, **kwargs) as process:
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 775, in __init__
    restore_signals, start_new_session)
  File "/home/suryak/anaconda3/lib/python3.7/subprocess.py", line 1453, in _execute_child
    restore_signals, start_new_session, preexec_fn)
OSError: [Errno 12] Cannot allocate memory


{'best_epoch': 1,
 'peak_cpu_memory_MB': 125591.588,
 'training_duration': '0:06:13.965647',
 'training_start_epoch': 0,
 'training_epochs': 1,
 'epoch': 1,
 'training_acc': 0.487263339070568,
 'training_loss': 3.4144281096510833,
 'training_cpu_memory_MB': 125591.588}

In [121]:
# inference
results = {}
test_iterator = BasicIterator(batch_size=1)
test_iterator.index_with(vocab)

sample = next(iter(test_iterator(dev_ds, shuffle=False)))
sample = move_to_device(sample, allen_device)

key = sample["id"][0]
output = model(**sample)

t_loc = int(output["target"].item())
p_loc = int(output["pred"].item())

target_val = sample["memory_values"][0][t_loc]
pred_val = sample["memory_values"][0][p_loc]

target_val, pred_val

('2', 'NONE')

In [142]:
def predictor(model, test_ds, device):
    results = defaultdict(OrderedDict)
    test_iterator = BasicIterator(batch_size=32)
    test_iterator.index_with(vocab)
    
    model = move_to_device(model, device)
    model = model.eval()
    
    for sample in tqdm(test_iterator(test_ds, shuffle=False, num_epochs=1)):
        sample = move_to_device(sample, device)
        with torch.no_grad():
            output = model(**sample)
        
        num_samples = output["target"].shape[0]
        for i in range(num_samples):
            key = sample["id"][i]
            t_loc = int(output["target"][i].item())
            p_loc = int(output["pred"][i].item())

            t_val = "UNK"
            p_val = "UNK"
            if t_loc < len(sample["memory_values"][i]):
                t_val = sample["memory_values"][i][t_loc]
            if p_loc < len(sample["memory_values"][i]):
                p_val = sample["memory_values"][i][p_loc]
        
            results[key] = (t_val, p_val, t_loc == p_loc)
    
    return results

In [144]:
results = predictor(model, dev_ds, allen_device)

In [153]:
pd.DataFrame(results.values(), index=results.keys())

,0,1,2
1_00000/0/Restaurants_2/number_of_seats,2,NONE,False
1_00000/0/Restaurants_2/time,UNK,NONE,False
1_00000/0/Restaurants_2/has_vegetarian_options,NONE,NONE,True
1_00000/0/Restaurants_2/date,NONE,NONE,True
1_00000/0/Restaurants_2/phone_number,NONE,NONE,True
1_00000/2/Restaurants_2/number_of_seats,2,NONE,False
1_00000/2/Restaurants_2/restaurant_name,UNK,NONE,False
1_00000/2/Restaurants_2/time,UNK,NONE,False
1_00000/2/Restaurants_2/location,UNK,UNK,True
1_00000/2/Restaurants_2/price_range,NONE,moderate,False
